# create stats for report

In [2]:
import sys
sys.path.append('../../')
import glob
import shutil
from pathlib import Path
import os

import numpy as np
import pandas as pd
from config import CLEAN_DATA_DIR, OUTPUT_DIR, RAW_DATA_DIR
import db as db
from scripts.normalize_taxa import add_normalized_name_column

from scripts.normalize_data import (
    check_duplicate_columns
)

from scripts.shared_utils import (
    create_df_from_db_rows, 
    log_df
)

In [3]:
clean_data_path = CLEAN_DATA_DIR

metadata_file = OUTPUT_DIR/'metadata'/'LIMS'/'Micropal_changes.csv' 
# metadata_file = OUTPUT_DIR/'metadata'/'LIMS'/'Micropal_changes_4.csv' 


date = '2022-11-15'

taxa_list_file = OUTPUT_DIR/'taxa'/'LIMS'/f"taxa_list_{date}.csv"
taxa_crosswalk_list_file = OUTPUT_DIR/'taxa'/'LIMS'/f"taxa_crosswalk_{date}.csv"

PI_file = RAW_DATA_DIR/'PI_processed_files'/f'LIMS_Micropal_headers_PBDB_Taxonomy_notes_taxa_list_{date}.csv'
PI_4_file = OUTPUT_DIR /'taxa'/'LIMS'/f'PI_Micropal_CSV_4_normalized_taxa_list_with_pbdb_{date}.csv'
additional_taxa_path = OUTPUT_DIR/'taxa'/'LIMS'/f'addtional_species_{date}.csv'
PI_NOAA_file = OUTPUT_DIR/'taxa'/'NOAA'/f'taxa_crosswalk_{date}.csv'

all_columns_file = OUTPUT_DIR/'tmp/all_LIMS_taxa_columns.csv'


## LIMS taxa stats

In [7]:
sql = """
select count(distinct  sections.core_id),
count (distinct sections.id),
count (distinct samples.id)

from sections 
join samples on samples.section_id = sections.id
where samples.data_source_type = 'micropal csv';

"""

row = db.fetch_one(sql)
print(row)
## 10,803 cores
## 19,775 sections
## 43,760 samples

[10803, 19775, 43760]


In [10]:


sql = """
select count(*)from taxa;
"""

row = db.fetch_one(sql)
print(  row['count'])


sql = """
select count(*)from taxa_crosswalk;
"""

row = db.fetch_one(sql)
print(  row['count'])


sql = """
select count(distinct taxon_group)from taxa;
"""

row = db.fetch_one(sql)
print(  row['count'])




4694
5380
13


In [32]:
sql = """
select  
taxa.name, taxa.taxon_group, taxa.pbdb_taxon_id,
array_to_string(array_agg(taxa_crosswalk.verbatim_name), '; ') as verbatim_name, 
array_to_string(array_agg(taxa_crosswalk.comment), '; ') as comment
from taxa 
join taxa_crosswalk on taxa.id = taxa_crosswalk.taxon_id
where   taxa.id in (select taxon_id from samples_taxa)
group by taxa.name, taxa.taxon_group , taxa.pbdb_taxon_id
order by taxa.name, taxa.taxon_group,  verbatim_name, comment ;
"""

rows = db.fetch_all_dict(sql)
len(rows)

# 4735

4736

In [33]:
df = create_df_from_db_rows(rows)
df['pbdb_taxon_id'] = df['pbdb_taxon_id'].astype(pd.Int64Dtype())
df.head()

,name,taxon_group,pbdb_taxon_id,verbatim_name,comment
0,"""Amorphous organic matter""",dinoflagellates,<NA>,Amorphous organic matter,not a taxa name
1,"""Amorphous organic matter""",palynology,<NA>,Amorphous organic matter,not a taxa name
2,"""Black phytoclasts""",dinoflagellates,<NA>,Black phytoclasts,not a taxa name
3,"""Black phytoclasts""",palynology,<NA>,Black phytoclasts,not a taxa name
4,"""Black woody phytoclasts""",palynology,<NA>,Black woody phytoclasts,not a taxa name


In [34]:
# df.to_csv(OUTPUT_DIR/'taxa'/'LIMS'/f'taxa_list_distinct_taxonomic_entries_{date}.csv', index=False)

In [30]:
sql = """
select count(distinct taxon_id) from samples_taxa;
"""

db.fetch_one(sql)[0]

4692

In [31]:
sql = """
select count(distinct(taxa.name, taxa.taxon_group))
from taxa 
where taxa.id in (select taxon_id from samples_taxa);
"""

db.fetch_one(sql)[0]

4692

In [32]:
sql = """
select count(distinct(taxa.name))
from taxa 
where taxa.id in (select taxon_id from samples_taxa);
"""

db.fetch_one(sql)[0]

4667

In [33]:
sql = """
select  
array_to_string(array_agg(taxa.taxon_group ORDER BY taxa.taxon_group), '; ') as taxon_group, 
array_to_string(array_agg(taxa_crosswalk.verbatim_name ORDER BY taxa_crosswalk.verbatim_name), '; ') as verbatim_name, 
taxa.name,
array_to_string(array_agg(taxa_crosswalk.comment ORDER BY taxa_crosswalk.comment), '; ') as comment,
taxon_name_above_genus,
genus_modifier, genus_name,
subgenera_modifier, subgenera_name,
species_modifier, species_name,
subspecies_modifier, subspecies_name,
non_taxa_descriptor,
array_to_string(array_agg(taxa_crosswalk.comments ORDER BY taxa_crosswalk.comments), '; ') as comments,
taxa.pbdb_taxon_id, taxa.pbdb_taxon_rank, taxa.pbdb_taxon_name
from taxa 
join taxa_crosswalk on taxa.id = taxa_crosswalk.taxon_id
where   taxa.id in (select taxon_id from samples_taxa)
group by taxa.name,  taxon_name_above_genus,
genus_modifier, genus_name,
subgenera_modifier, subgenera_name,
species_modifier, species_name,
subspecies_modifier, subspecies_name,
non_taxa_descriptor,  
taxa.pbdb_taxon_id, taxa.pbdb_taxon_rank, taxa.pbdb_taxon_name
order by taxa.name,   verbatim_name, comment ;
"""




rows = db.fetch_all_dict(sql)
len(rows)

# 4667

4667

In [34]:
df = create_df_from_db_rows(rows)
df['pbdb_taxon_id'] = df['pbdb_taxon_id'].astype(pd.Int64Dtype())
df.tail()

,taxon_group,verbatim_name,name,comment,taxon_name_above_genus,genus_modifier,genus_name,subgenera_modifier,subgenera_name,species_modifier,species_name,subspecies_modifier,subspecies_name,non_taxa_descriptor,comments,pbdb_taxon_id,pbdb_taxon_rank,pbdb_taxon_name
4662,nannofossils,Zygodiscus plectopons,Zygodiscus plectopons,<NA>,<NA>,<NA>,Zygodiscus,<NA>,<NA>,<NA>,plectopons,<NA>,<NA>,<NA>,<NA>,87678,genus,Zygodiscus
4663,nannofossils,Zygrhablithus bijugatus,Zygrhablithus bijugatus,<NA>,<NA>,<NA>,Zygrhablithus,<NA>,<NA>,<NA>,bijugatus,<NA>,<NA>,<NA>,<NA>,416732,species,Zygrhablithus bijugatus
4664,radiolarians,Spongurus cf. elliptica,cf. Spongurus elliptica,<NA>,<NA>,cf.,Spongurus,<NA>,<NA>,<NA>,elliptica,<NA>,<NA>,<NA>,<NA>,328,genus,Spongurus
4665,radiolarians,Thyrsocyrtis cf. norrisi,cf. Thyrsocyrtis norrisi,<NA>,<NA>,cf.,Thyrsocyrtis,<NA>,<NA>,<NA>,norrisi,<NA>,<NA>,<NA>,<NA>,685,genus,Thyrsocyrtis
4666,radiolarians,Velicucullus sp. cf. oddgurneri,cf. Velicucullus oddgurneri,<NA>,<NA>,cf.,Velicucullus,<NA>,<NA>,<NA>,oddgurneri,<NA>,<NA>,<NA>,<NA>,705,genus,Velicucullus


In [35]:
df.to_csv(OUTPUT_DIR/'taxa'/'LIMS'/f'taxa_list_distinct_taxonomic_entries_across_groups_{date}.csv', index=False)

In [94]:
sql = """
select distinct samples.raw_data ->> 'Exp' as name, data_source_notes,
data_source_type
from samples
where name is not NULL
"""

rows = db.fetch_all_dict(sql)
df = pd.DataFrame(rows)
log_df(df)

(1769, 3)


,name,data_source_notes,data_source_type
0,317,317_Lithostratigraphy_U1351A.csv,lithology csv
1,317,317_Lithostratigraphy_U1351B.csv,lithology csv
2,317,317_Lithostratigraphy_U1352A.csv,lithology csv
3,317,317_Lithostratigraphy_U1352B.csv,lithology csv
4,317,317_Lithostratigraphy_U1352C.csv,lithology csv


In [95]:
pivot = pd.pivot_table(df, index=['name'], columns=['data_source_type'], values=['data_source_type'], aggfunc='count')
log_df(pivot)

(38, 2)


data_source_notes             
data_source_type     lithology csv micropal csv
name                                           
317                           11.0         16.0
318                            NaN         52.0
320                           16.0         82.0
321                            NaN         15.0
323                           24.0        132.0

In [96]:
pivot.to_csv(OUTPUT_DIR/'taxa'/'LIMS'/f'expeditions_2022-12-02.csv')